<a href="https://colab.research.google.com/github/zeylulbalci/dsa210project/blob/main/TMDB_Movies_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Step 1: Load your main dataset
df_main = pd.read_csv("IMDb___Gross_Dataset.csv")

# Clean titles and ensure year is numeric
df_main['clean_title'] = df_main['Movie'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
df_main['Year'] = pd.to_numeric(df_main['Year'], errors='coerce')

# Step 2: Define columns to keep from TMDB dataset
columns_to_keep = ['title', 'release_date', 'budget', 'revenue', 'runtime', 'genres', 'popularity', 'vote_average', 'vote_count']
chunksize = 100_000
matched_rows = []

# Step 3: Read TMDB in chunks (safe read with error skipping)
for chunk in pd.read_csv(
    "TMDB_movie_dataset_v11.csv",
    usecols=columns_to_keep,
    chunksize=chunksize,
    encoding='utf-8',
    on_bad_lines='skip',
    low_memory=False
):
    chunk = chunk.copy()  # Avoid SettingWithCopyWarning

    # Preprocess TMDB chunk
    chunk['clean_title'] = chunk['title'].astype(str).str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()
    chunk['release_date'] = pd.to_datetime(chunk['release_date'], errors='coerce')
    chunk = chunk.dropna(subset=['release_date'])
    chunk['release_year'] = chunk['release_date'].dt.year.astype(int)

    # Merge on clean_title + year
    merged = chunk.merge(df_main, how='inner', left_on=['clean_title', 'release_year'], right_on=['clean_title', 'Year'])
    matched_rows.append(merged)

# Step 4: Concatenate all matched chunks
df_matched = pd.concat(matched_rows, ignore_index=True)

# Step 5: Select and rename relevant columns
df_final = df_matched[[
    'Movie', 'Year', 'budget', 'revenue', 'runtime', 'genres', 'popularity',
    'vote_average', 'vote_count', 'Views', 'Likes', 'Comments',
    'IMDb_Rating', 'numVotes', 'Gross'
]]

# Step 6: Save final enriched dataset
df_final.to_csv("Enriched_Movie_Dataset.csv", index=False)
print("✅ Done! Enriched dataset created with", df_final.shape[0], "matched rows.")


<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['release_year'] = chunk['release_date'].dt.year.astype(int)
<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['release_year'] = chunk['release_date'].dt.year.astype(int)
<ipython-input-1-4da5be5e1d7b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

✅ Done! Enriched dataset created with 280 matched rows.


In [3]:
import pandas as pd
import numpy as np

# Load the enriched dataset
df = pd.read_csv("Enriched_Movie_Dataset.csv")

# ⚠️ Prevent division by zero in later ratio calculations
df['Views'] = df['Views'].replace(0, np.nan)
df['budget'] = df['budget'].replace(0, np.nan)

# --- FEATURE ENGINEERING BEGINS HERE ---

# ▶️ 1. Engagement Ratios
# These features measure how interactive or engaging the trailer was
df['LikeRatio'] = df['Likes'] / df['Views']           # Higher ratio = more approval per view
df['CommentRatio'] = df['Comments'] / df['Views']     # Higher ratio = more discussion/interest
df['VoteRatio'] = df['numVotes'] / df['Views']        # IMDb engagement vs YouTube exposure

# ▶️ 2. Financial Metrics
# These help capture economic success more accurately than raw revenue
df['Profit'] = df['revenue'] - df['budget']           # Actual money made
df['ROI'] = df['revenue'] / df['budget']              # Efficiency of budget use

df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')


# ▶️ 3. Log Transformations
# Gross and revenue tend to be highly skewed; log transformation normalizes them for modeling
df['LogGross'] = np.log1p(df['Gross'])
df['LogBudget'] = np.log1p(df['budget'])
df['LogRevenue'] = np.log1p(df['revenue'])

# ▶️ 4. Release Month
# Adds seasonal information that could affect box office success
if 'release_date' in df.columns:
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    df['release_month'] = df['release_date'].dt.month
else:
    df['release_month'] = np.nan  # Placeholder if release date not available

# ▶️ 5. Genre Flags
# Converts multi-category text into binary variables usable in ML models
df['is_action'] = df['genres'].astype(str).str.contains('Action', case=False, na=False).astype(int)
df['is_comedy'] = df['genres'].astype(str).str.contains('Comedy', case=False, na=False).astype(int)
df['is_drama'] = df['genres'].astype(str).str.contains('Drama', case=False, na=False).astype(int)

# ▶️ 6. Runtime Category
# Groups continuous runtime into discrete categories for interpretable modeling
df['runtime_category'] = pd.cut(df['runtime'],
                                bins=[0, 80, 120, 200],
                                labels=['Short', 'Medium', 'Long'])

# ⚠️ 7. Clean up: Remove rows with invalid values generated during ratio calculations
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[
    'LikeRatio', 'CommentRatio', 'VoteRatio', 'Profit', 'ROI'
])

# 💾 Save the final dataset
df.to_csv("Final_Engineered_Dataset.csv", index=False)

# ✅ Summary Comment:
# After this step, we have a dataset enriched with over 10 new engineered features,
# including financial ratios, engagement metrics, normalized values,
# and categorical expansions that make the dataset more suitable for ML modeling.
print("✅ Feature engineering completed. Final shape:", df.shape)


✅ Feature engineering completed. Final shape: (92, 28)


In [6]:
import pandas as pd
import numpy as np

# Load the merged dataset (from TMDB + your base dataset)
df = pd.read_csv("Enriched_Movie_Dataset.csv")  # <- kendi dosya adını koy

# Step 1: Sayısal sütunları güvenli şekilde sayıya çevir
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['Views'] = pd.to_numeric(df['Views'], errors='coerce')

# Step 2: Sadece kritik değişkenler eksikse sil
df_cleaned = df.dropna(subset=['revenue', 'budget', 'runtime', 'Views'])

# Step 3: Oranlar (ratio) hesapla
df_cleaned['LikeRatio'] = df_cleaned['Likes'] / df_cleaned['Views']
df_cleaned['CommentRatio'] = df_cleaned['Comments'] / df_cleaned['Views']
df_cleaned['VoteRatio'] = df_cleaned['numVotes'] / df_cleaned['Views']

# Oranlardaki eksikleri sıfırla
df_cleaned[['LikeRatio', 'CommentRatio', 'VoteRatio']] = df_cleaned[[
    'LikeRatio', 'CommentRatio', 'VoteRatio'
]].fillna(0)

# Step 4: Finansal metrikler
df_cleaned['Profit'] = df_cleaned['revenue'] - df_cleaned['budget']
df_cleaned['ROI'] = df_cleaned['revenue'] / df_cleaned['budget']

# Step 5: Log dönüşümler
df_cleaned['LogGross'] = np.log1p(df_cleaned['Gross'])
df_cleaned['LogBudget'] = np.log1p(df_cleaned['budget'])
df_cleaned['LogRevenue'] = np.log1p(df_cleaned['revenue'])

# Step 6: Genre flags (isteğe bağlı örnekler)
df_cleaned['is_action'] = df_cleaned['genres'].astype(str).str.contains('Action', case=False, na=False).astype(int)
df_cleaned['is_comedy'] = df_cleaned['genres'].astype(str).str.contains('Comedy', case=False, na=False).astype(int)
df_cleaned['is_drama'] = df_cleaned['genres'].astype(str).str.contains('Drama', case=False, na=False).astype(int)

# Step 7: Runtime kategorisi
df_cleaned['runtime_category'] = pd.cut(df_cleaned['runtime'],
                                        bins=[0, 80, 120, 200],
                                        labels=['Short', 'Medium', 'Long'])

# Step 8: Logik hatalar ve ∞ değerleri temizle
df_cleaned = df_cleaned.replace([np.inf, -np.inf], np.nan)

# Final dataframe hazır
df_cleaned.to_csv("Final_Engineered_Dataset.csv", index=False)
print("✅ Final dataset is ready. Shape:", df_cleaned.shape)


✅ Final dataset is ready. Shape: (280, 27)
